# ML Pipeline Preparation
Create the ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize, sent_tokenize

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report


In [11]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
#df = pd.read_sql_table('SELECT * FROM DisasterResponse', engine)
df = pd.read_csv('DisasterResponse.csv', dtype = {'message': str, 'original': str, 'genre': str})
X = df["message"]
y = df.iloc[:, 4:]

In [4]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [5]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    return word_tokenize(text)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator = RandomForestClassifier(random_state = 42)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred = pd.DataFrame(data = pipeline.predict(X_test), columns = y.columns)

In [12]:
precision, recall, f1_score = [], [], []

for category in y.columns:
    scores = classification_report(y_test[category], y_pred[category])
    precision.append([x for x in scores.strip().split("avg / total")[1].strip().split(" ") if len(x) > 0][:3][0])
    recall.append([x for x in scores.strip().split("avg / total")[1].strip().split(" ") if len(x) > 0][:3][1])
    f1_score.append([x for x in scores.strip().split("avg / total")[1].strip().split(" ") if len(x) > 0][:3][2])    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [34]:
classification_metric = pd.DataFrame(data = [precision, recall, f1_score], 
                                     index = ["precision", "recall", "f1_score"], columns = y.columns)#.T

classification_metric

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
precision,0.73,0.89,0.99,0.74,0.91,0.95,0.97,0.97,0.96,1.00,...,0.98,0.93,0.87,0.95,0.93,0.99,0.97,0.98,0.93,0.85
recall,0.77,0.90,0.99,0.74,0.92,0.96,0.98,0.98,0.97,1.00,...,0.99,0.95,0.88,0.96,0.94,0.99,0.97,0.98,0.95,0.86
f1_score,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=None, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=42, verbose=0, warm_start=False),
    

In [15]:
def build_model():
    
    pipeline = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(estimator = RandomForestClassifier(random_state = 42)))
    ])
    
    
    parameters = {"clf__estimator__min_samples_leaf": [0.05, 0.10, 0.15],
              "clf__estimator__criterion": ["gini", "entropy"],
              "clf__estimator__max_depth": range(3,20,5)
              }
    
    from sklearn.model_selection import GridSearchCV
    cv = GridSearchCV(pipeline, param_grid = parameters)
    
    return cv

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
model = build_model()
model.fit(X_train, y_train)
model.best_params_

{'clf__estimator__criterion': 'gini',
 'clf__estimator__max_depth': 3,
 'clf__estimator__min_samples_leaf': 0.05}

In [17]:
y_pred = pd.DataFrame(data = model.predict(X_test), columns = y.columns)

In [18]:
precision, recall, f1_score = [], [], []

for category in y.columns:
    scores = classification_report(y_test[category], y_pred[category])
    precision.append([x for x in scores.strip().split("avg / total")[1].strip().split(" ") 
                      if len(x) > 0][:3][0])
    recall.append([x for x in scores.strip().split("avg / total")[1].strip().split(" ") 
                   if len(x) > 0][:3][1])
    
optimized_metric = pd.concat([
    pd.DataFrame(data = [precision, recall], index = ["precision", "recall"], 
                 columns = y.columns),
    (y_test.reset_index() == y_pred.reset_index()).mean()[1:].to_frame("accuracy").T
])
    
optimized_metric    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
precision,0.57,0.69,0.99,0.35,0.84,0.90,0.95,0.96,0.94,1.00,...,0.98,0.92,0.53,0.85,0.82,0.98,0.83,0.96,0.90,0.65
recall,0.75,0.83,0.99,0.59,0.92,0.95,0.98,0.98,0.97,1.00,...,0.99,0.96,0.73,0.92,0.91,0.99,0.91,0.98,0.95,0.81
accuracy,0.754348,0.830485,0.994965,0.592615,0.91837,0.947513,0.97574,0.982301,0.969484,1,...,0.987641,0.95682,0.729478,0.920812,0.90769,0.98993,0.909979,0.978486,0.948886,0.80592


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [23]:
def build_model_v2():
    
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.pipeline import Pipeline, FeatureUnion
    
    pipeline = Pipeline([
        ('features', FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer())
            ])),
            
            ('strarting_verb', StartingVerbExtractor())
            
        ])),

         ('clf', MultiOutputClassifier(estimator = AdaBoostClassifier(random_state = 42)))

    ])
        
    return pipeline

In [24]:
from sklearn.base import BaseEstimator, TransformerMixin
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    
    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False
    
    def fit(self, x, y=None):
        return self
    
    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [27]:
model_ada = build_model_v2()
model_ada.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...timator=None,
          learning_rate=1.0, n_estimators=50, random_state=42),
           n_jobs=1))])

In [28]:
model_ada

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...timator=None,
          learning_rate=1.0, n_estimators=50, random_state=42),
           n_jobs=1))])

In [29]:
y_pred = pd.DataFrame(data = model_ada.predict(X_test), columns = y.columns)

precision, recall, f1_score = [], [], []

for category in y.columns:
    scores = classification_report(y_test[category], y_pred[category])
    precision.append([x for x in scores.strip().split("avg / total")[1].strip().split(" ") 
                      if len(x) > 0][:3][0])
    recall.append([x for x in scores.strip().split("avg / total")[1].strip().split(" ") 
                   if len(x) > 0][:3][1])
    
ada_metric_v2 = pd.concat([
    pd.DataFrame(data = [precision, recall], index = ["precision", "recall"], 
                 columns = y.columns),
    (y_test.reset_index() == y_pred.reset_index()).mean()[1:].to_frame("accuracy").T
])

In [30]:
ada_metric_v2    

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
precision,0.73,0.89,0.99,0.74,0.91,0.95,0.97,0.97,0.96,1.00,...,0.98,0.93,0.87,0.95,0.93,0.99,0.97,0.98,0.93,0.85
recall,0.77,0.90,0.99,0.74,0.92,0.96,0.98,0.98,0.97,1.00,...,0.99,0.95,0.88,0.96,0.94,0.99,0.97,0.98,0.95,0.86
accuracy,0.765334,0.89762,0.994202,0.744889,0.923711,0.956363,0.975282,0.980012,0.970095,1,...,0.985505,0.951938,0.876717,0.957431,0.935307,0.99115,0.972078,0.982759,0.946903,0.859017


In [32]:
# https://github.com/epmoyer/ipy_table/issues/24   

from IPython.core.display import HTML

def multi_table(table_list):
    ''' Acceps a list of IpyTable objects and returns a table which contains each IpyTable in a cell
    '''
    return HTML(
        '<table><tr style="background-color:white;">' + 
        ''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list]) +
        '</tr></table>'
    )

In [35]:
multi_table([classification_metric.T, optimized_metric.T, ada_metric_v2.T])

,precision,recall,f1_score
related,0.73,0.77,None
request,0.89,0.90,None
offer,0.99,0.99,None
aid_related,0.74,0.74,None
medical_help,0.91,0.92,None
medical_products,0.95,0.96,None
search_and_rescue,0.97,0.98,None
security,0.97,0.98,None
military,0.96,0.97,None
child_alone,1.00,1.00,None


In [42]:
for table in [classification_metric, optimized_metric, ada_metric_v2]:
    for col in table.columns:
        table[col] = table[col].astype(float)

In [47]:
multi_table([
    classification_metric.T.mean().to_frame("mean"), 
    optimized_metric.T.mean().to_frame("mean"), 
    ada_metric_v2.T.mean().to_frame("mean")
])

,mean
precision,0.938889
recall,0.947778
f1_score,NaN
,mean
precision,0.867500
recall,0.927778
accuracy,0.927212
,mean
precision,0.938889
recall,0.947778


The pipeline with Adaboost has better precision and accuracy. /

### 9. Export your model as a pickle file

In [48]:
import pickle

pickle.dump(model_ada, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.